# Testing code for functions!

In [ ]:
from modules.module_for_imports import *

In [ ]:
#try2 = clean_clubs(get_clubs())
#try2.info()

In [ ]:
clubs = get_clubs()
players = get_players()

In [ ]:
squad_sizes = players.groupby('current_club_id')['player_id'].count()
squad_sizes = pd.DataFrame(squad_sizes)
squad_sizes.rename(columns={'current_club_id': 'club_id', 'player_id': 'size'}, inplace=True)
squad_sizes

In [ ]:
# to be sure it's right:
players.query('current_club_id == 3').shape

In [ ]:
merged_df = pd.merge(clubs, squad_sizes, left_on='club_id', right_on='current_club_id', how='left')

merged_df.query('squad_size > 10 & squad_size != size')

> # How should we proceed?! We have different sizes in few cases!